In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 36.3 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/WMT_Grocery_202209.csv")
df

,DEPARTMENT,CATEGORY,PRODUCT_NAME
0,Deli,"Hummus, Dips, & Salsa","Marketside Roasted Red Pepper Hummus, 10 Oz"
1,Deli,"Hummus, Dips, & Salsa","Marketside Roasted Garlic Hummus, 10 Oz"
2,Deli,"Hummus, Dips, & Salsa","Marketside Classic Hummus, 10 Oz"
3,Deli,"Hummus, Dips, & Salsa","Marketside Everything Hummus, 10 oz"
4,Deli,"Hummus, Dips, & Salsa","Price's Jalapeno Dip, 12 Oz."
...,...,...,...
568529,Alcohol,Wine,Farm Fresh Blueberry Moscato 750ml
568530,Alcohol,Wine,Farm Fresh Peach Moscato 750 Ml
568531,Alcohol,Wine,Farm Fresh Raspberry Moscato 750ml
568532,Alcohol,Wine,Farm Fresh Mango Moscato 750ml


In [ ]:
df["new"]=df["PRODUCT_NAME"]+" =>:"+df["CATEGORY"]
df.head()

,DEPARTMENT,CATEGORY,PRODUCT_NAME,new
0,Deli,"Hummus, Dips, & Salsa","Marketside Roasted Red Pepper Hummus, 10 Oz","Marketside Roasted Red Pepper Hummus, 10 Oz =>..."
1,Deli,"Hummus, Dips, & Salsa","Marketside Roasted Garlic Hummus, 10 Oz","Marketside Roasted Garlic Hummus, 10 Oz =>:Hum..."
2,Deli,"Hummus, Dips, & Salsa","Marketside Classic Hummus, 10 Oz","Marketside Classic Hummus, 10 Oz =>:Hummus, Di..."
3,Deli,"Hummus, Dips, & Salsa","Marketside Everything Hummus, 10 oz","Marketside Everything Hummus, 10 oz =>:Hummus,..."
4,Deli,"Hummus, Dips, & Salsa","Price's Jalapeno Dip, 12 Oz.","Price's Jalapeno Dip, 12 Oz. =>:Hummus, Dips, ..."


In [ ]:
df["new"]

0         Marketside Roasted Red Pepper Hummus, 10 Oz =>...
1         Marketside Roasted Garlic Hummus, 10 Oz =>:Hum...
2         Marketside Classic Hummus, 10 Oz =>:Hummus, Di...
3         Marketside Everything Hummus, 10 oz =>:Hummus,...
4         Price's Jalapeno Dip, 12 Oz. =>:Hummus, Dips, ...
                                ...                        
568529           Farm Fresh Blueberry Moscato 750ml =>:Wine
568530              Farm Fresh Peach Moscato 750 Ml =>:Wine
568531           Farm Fresh Raspberry Moscato 750ml =>:Wine
568532               Farm Fresh Mango Moscato 750ml =>:Wine
568533          Ole Orleans Heritage Riesling 750ml =>:Wine
Name: new, Length: 568534, dtype: object

# Loading the base model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences = pipeline(
   ["“Marketside Ceasar salad” =>:","Frozen Wild Caught Mahi-Mahi Fillet =>:","Farm Fresh Raspberry Moscato 750ml =>:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"{seq[0]['generated_text']}")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


“Marketside Ceasar salad” =>:
 Einzeln: $0.79 (was: $0.89)

:

* “Marketside Chicken Caesar salad with croutons & parmesans” =>: Einzeln: $2.29 (was: $2.49)
* “Marketside Chicken salad with croutons, bacon and parmesan shards” =>: Einzeln: $2.34 (was: $2.50)
* “Marketsided Chicken salad with croutons, bacon & parmesan shards” =>: Einzeln: $2.39 (was: $2.49)
* “Marketside Chicken Caesar salad with croutons and parmesan shards” =>: Einzeln: $2.49 (was: $2.69)
* “
Frozen Wild Caught Mahi-Mahi Fillet =>: https://www. Einzelnes: Mahi-mahi. Mahi Mahi. Mahi-mahi is a fish with a delicious, mild flavor and a firm texture that is similar to swordfish and tuna. 8 oz 4.99 / 34 oz. 1. Mahi-mahi is a tropical fish that is related to dolphinfish, and is found in warm and temperate waters around the world. It is known as Dorado in some parts, and Dorade or Golden Dorado in the Caribbean. The mahi-mahi is one of the most delicious fish out there, but how do you prepare it? Fresh fish, filleting, and p

# Injecting LoRA config to the default model

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

# Now only .06 parameter needs to be trained for this text generation task.

In [ ]:
from peft import LoraConfig, get_peft_model

peft_model = get_peft_model(model, peft_config)
print(peft_model.print_trainable_parameters())

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
df

,DEPARTMENT,CATEGORY,PRODUCT_NAME,new
0,Deli,"Hummus, Dips, & Salsa","Marketside Roasted Red Pepper Hummus, 10 Oz","Marketside Roasted Red Pepper Hummus, 10 Oz =>..."
1,Deli,"Hummus, Dips, & Salsa","Marketside Roasted Garlic Hummus, 10 Oz","Marketside Roasted Garlic Hummus, 10 Oz =>:Hum..."
2,Deli,"Hummus, Dips, & Salsa","Marketside Classic Hummus, 10 Oz","Marketside Classic Hummus, 10 Oz =>:Hummus, Di..."
3,Deli,"Hummus, Dips, & Salsa","Marketside Everything Hummus, 10 oz","Marketside Everything Hummus, 10 oz =>:Hummus,..."
4,Deli,"Hummus, Dips, & Salsa","Price's Jalapeno Dip, 12 Oz.","Price's Jalapeno Dip, 12 Oz. =>:Hummus, Dips, ..."
...,...,...,...,...
568529,Alcohol,Wine,Farm Fresh Blueberry Moscato 750ml,Farm Fresh Blueberry Moscato 750ml =>:Wine
568530,Alcohol,Wine,Farm Fresh Peach Moscato 750 Ml,Farm Fresh Peach Moscato 750 Ml =>:Wine
568531,Alcohol,Wine,Farm Fresh Raspberry Moscato 750ml,Farm Fresh Raspberry Moscato 750ml =>:Wine
568532,Alcohol,Wine,Farm Fresh Mango Moscato 750ml,Farm Fresh Mango Moscato 750ml =>:Wine


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(df),
})

# Training LoRA

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict["train"],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="new",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/568534 [00:00<?, ? examples/s]

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.390300
2,1.445600
3,2.765900
4,1.940700
5,2.736400
6,2.645300
7,2.514300
8,2.198300
9,3.121000
10,2.194700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=120, training_loss=3.9038525452216466, metrics={'train_runtime': 585.9775, 'train_samples_per_second': 3.277, 'train_steps_per_second': 0.205, 'total_flos': 1302441962766336.0, 'train_loss': 3.9038525452216466, 'epoch': 0.0})

In [ ]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences = pipeline(
   ["“Marketside Ceasar salad” =>:","Frozen wild caught fillet =>:","French Milled Oval Almond Gourmande Soap =>:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Result: “Marketside Ceasar salad” =>:Deli Salads & Desserts:Deli Salads and Desserts & Fresh Fruit & Vegetable Platters & Trays & Sides & Appetizers & Sausage & Hot Dogs & Cheese ::Food to Go ::Frozen Foods & Beverages =>:Deli Salads & Desserts ::Garden salads & Deli Salads =>:Salads & Salad Dressings ::Fresh Fruit =>:Salad Dressings, Dips & Sauces =>:Salad Dressing & Sauces =>:Dressings:Reduced Fat =>:Dressings, Sauces & Salsas =>:Salad Dressings =>:Caesar Salad Dressing =>:Marketside Caesar Salad Dressing - 10. =>:
Result: Frozen wild caught fillet =>:Wild Caught Seafood & Sushi Meats :Fish & Seafood:Fresh Fish & Seafood:Frozen Seafood:Frozen Seafood & Fish =>:Frozen Fish Fillets =>:Frozen Seafood Stuffed Fish & Seafood =>:Frozen Canned or Frozen Seafood & Crab =>:Seafood =>:Fresh Fish & Seafoods & Sushi =>:Seafood =>:Wild Caught Fish:Mackerel =>:Fresh Fish & Seafoods & Sushi =>:Fresh Canned or Fresh Frozen Seafood & Crab =>:Fresh Seafood & Crab =>:Cod & Haddock & Cod Fillets =>:Fres

In [ ]:
for pred in prediction_lora:
    print(f"{pred[0]['generated_text']}")

[[{'generated_text': '“Marketside Ceasar salad” =>:Hummus Dips & Salsa'}],
[{'generated_text': 'Frozen Wild Caught Mahi-Mahi Fillet =>:Seafood & Sushi Meats'}],
[{'generated_text': 'Farm Fresh Raspberry Moscato 750ml =>:Wine'}]]
